Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name below:

In [ ]:
NAME = "Munzir H. Abdulmajid"

---

ME 535 Winter 2021

# Homework 2

### Thursday Feb. 4

1) This problem involves writing and using an alternate implementation of elementary row operations. The implementation in the Ch. 2 notebook uses a `for` loop to individually update each entry in the row being altered. Here you should employ an alternate notation supported by numpy that provides a way to refer to a portion of an array including an entire row or column.

The related keywords in numpy are "views", "slice", and "copy". Please read the materials at the following link that provides a nice description of the details:
https://jakevdp.github.io/PythonDataScienceHandbook/02.02-the-basics-of-numpy-arrays.html

1a) Write a new version of `row_op` that operates row-wise instead of element-wise.

In [1]:
import numpy as np

def row_op(A,c,i0,i1):
    """
    perform elementary row operations
    if i1==i2, multiply row by constant c
    if i1!=i2, add c*(row i1) to row i2
    
    Args:
        A: 2D numpy array representing a matrix
        c: multiplicative constant
        i1,i2: row indices
    """
# YOUR CODE HERE
    m,n = A.shape
    if  i0<0 or i1<0 or i0>=m or i1>=m:
        print("WARNING: Invalid index specifications. Each index value i must satisfy 0<=i<#rows.")
    if i0==i1: #repeated index -> multiply row by constant
        for j in range(n):
            A[i0,j] *= c #A[i1,j] = c*A[i1,j]
    else: # add c*row i1 to row i2
        for j in range(n):
            A[i1,j] += c * A[i0,j]   


In [2]:
from numpy.testing import assert_, assert_raises

mat = np.eye(3)
row_op(mat,2,1,1)
ans = np.array([[1,0,0],[0,2,0],[0,0,1]], dtype = np.float64)
assert_(np.allclose(mat, ans))

mat = np.eye(3)
row_op(mat,3,1,2)
ans = np.array([[1,0,0],[0,1,0],[0,3,1]], dtype = np.float64)
assert_(np.allclose(mat, ans))

# TESTS

#assert()

1b) Previously we skipped one of the elementary row operations; i.e. swapping rows. Now is the time to fill in that gap. Write code for a row-wise implementation of `swap_rows(A, i0, i1)` that exchanges rows `i0` and `i1` in the array `A`.

In [3]:
def swap_rows(A,i0,i1):
    """
    perform elementary row operation to swap rows i0 and i1
    
    Args:
        A: 2D numpy array representing a matrix
        i0,i1: integer row indices
    """
# YOUR CODE HERE
    A[[i0, i1]] = A[[i1, i0]]

In [4]:
mat = np.eye(3)
swap_rows(mat,0,1)
ans = np.array([[0,1,0],[1,0,0],[0,0,1]], dtype = np.float64)
assert_(np.allclose(mat, ans))

2) We saw that elementary row operations involved in Gaussian elimination can be implemented in terms of $LU$ matrix factorization. However, not all matrices have an $LU$ decomposition. In fact, not all square non-singular matrices have an $LU$ decomposition. Here is perhaps the simplest classic example (from the book by VanLoan.

$\begin{bmatrix} 0 & 1 \\ 1 & 1 \end{bmatrix}
= 
\begin{bmatrix} 1 & 0 \\ l_{10} & 1 \end{bmatrix}
\begin{bmatrix} u_{00} & u_{01} \\ 0 & u_{11} \end{bmatrix}
=
\begin{bmatrix} u_{00} & u_{01} \\ l_{10} u_{00} & u_{00} + u_{11} \end{bmatrix}$

$\implies u_{00} = 0$ and $l_{10} u_{00} = l_{10} \cdot 0 = 1$ which cannot be satisfied by any finite value of $l_{10}$, so this matrix does not have an $LU$ factorization.

Note that giving the upper left entry a small non-zero value $\delta$ is not sufficient to fix the problem with trying to apply an $LU$ solver. For example, considering the following system which has the exact solution $x = [1, \, 1] ^T$:

$A_{\delta}x =
\begin{bmatrix} \delta & 1 \\ 1 & 1 \end{bmatrix}
\begin{bmatrix} x_0 \\ x_1 \end{bmatrix}
= 
\begin{bmatrix} 1 + \delta \\ 2 \end{bmatrix}
= b_{\delta}$

Use the $LU$ solver code included in the cell below to solve this problem for $x$ with $\delta = 10^{-k}$ for `k in range(5, 20, 2)` using 32-bit floats. Print out a table with the values of $\delta, x_0, x_1$. (Creating your numpy arrays with the optional argument `dtype = np.float32` should lead to 32 bit precision instead of 64 bit precision.)

In [5]:
#LU SOLVER CODE

def upper_tri_solve(U,b):
    m,n = U.shape #matrix has m rows and n columns
    x=np.zeros(m) #create an array to store the solution (init to zeros)
    for i in range(m):
        row = m-1-i #loop over rows from m-1 bacwards to 0
        accum=0 #variable to store sum of coeffs times known entries in solution
        for j in range(row+1,m): #loop over the part of row to right of diagonal
            accum+=U[row,j]*x[j]
        x[row]=(b[row]-accum)/U[row,row] #solve for i^th entry in solution
    return x

def lower_tri_solve(L,b):
    m,n = L.shape #matrix has m rows and n columns
    # should really check for compatible size
    y=np.zeros(m) #create an array to store the solution (init to zeros)
    for i in range(m):
        row = i
        accum=0
        for j in range(i):           
            accum+=L[row,j]*y[j]
        y[row]=(b[row]-accum)/L[i,i] #solve for i^th entry in solution
    return y

def LU_factor(A):
    m,n = A.shape
    if m != n:
        print("WARNING: Non-square input matrix")
        return
    mult = 0
    U = np.copy(A) #make a copy of the array
    #Note that U=A just makes another name for A, not a new copy of the array
    L = np.eye(n) #numpy's name for the identity matrix is "eye"
    for i in range(n): # for each row i
        for j in range(i+1,n): # for each row below row i
            mult = U[j,i]/U[i,i]
            L[j,i] = mult
            for k in range(i,n): # for each entry beyond the i^th diagonal entry
                U[j,k] = U[j,k] - mult*U[i,k] # for entries to the right, subtract multiple of term in row i         
    return L,U

def LU_solve(A,b):
    L, U = LU_factor(A)
    y = lower_tri_solve(L,b)
    x = upper_tri_solve(U,y)
    return x, y

In [6]:
# YOUR CODE HERE
A = np.array([[1,1],[1,1]], dtype=np.float32)
b = np.array([1,2], dtype=np.float32)
print(" Delta                       x(0)                         x(1) " )
print("--------------------------------------------------------------")
for k in range(5,20,2):
    delta = 10**(-k)
    A[0,0] = delta
    b[0] =  1 + delta
    x,y = LU_solve(A,b)
    print("%20.15f %20.15f %20.15f" %(delta, x[0], x[1]))

 Delta                       x(0)                         x(1) 
--------------------------------------------------------------
   0.000010000000000    1.000000011679546    1.000000013580458
   0.000000100000000    0.999999968913756    1.000000019209291
   0.000000001000000    2.000000111022306    0.999999998000000
   0.000000000010000    2.000000173472348    0.999999999980000
   0.000000000000100    1.999511702437998    0.999999999999800
   0.000000000000001    1.998401437076093    0.999999999999998
   0.000000000000000    0.000000000000000    1.000000000000000
   0.000000000000000    0.000000000000000    1.000000000000000


In the cells below, describe how the accuracy of the solution changes as $\delta$ becomes small.

In [58]:
# YOUR CODE HERE
print("As δ becomes small the accuracy increases. However, as δ becomes too small the floating point arithametic returns zero. Therefore, delta shouldn't be too small")

As δ becomes small the accuracy increases. However, as δ becomes too small the floating point arithametic returns zero. Therefore, delta shouldn't be too small


What is the value `k6` of the decimal exponent `k` is the solution no longer accurate to at least 6 digits?

In [59]:
# assign the appropriate value for the variable k6
# YOUR CODE HERE
print("k must be greater than/equal to 1e-7 for the 6th digit to be accurate. At k6 = 1e-9 It starts being unaccurate") 

k must be greater than/equal to 1e-7 for the 6th digit to be accurate. At k6 = 1e-9 It starts being unaccurate


Swap the rows in $A_{\delta}x = b_{\delta}$ and again apply the $LU$ solver to produce a corresponding table of results. 

Answer `true` or `false` (by assigning that value to the variable `ans` in the cell below:
Do you see a similar degradation of precision as $\delta$ becomes small?

In [7]:
# solve and print table
A = np.array([[1,1],[0,1]], dtype=np.float32)
b = np.array([2,1], dtype=np.float32)
print(" Delta         x(0)                 x(1) " )
print("-------------------------------------------------")
for k in range(5,19,2):
    delta = 10**(-k)
    A[1,0] = delta
    b[1] =  1 + delta
    x,y = LU_solve(A,b)
    print("%5.0e %20.15f %20.15f" %(delta, x[0], x[1]))

 Delta         x(0)                 x(1) 
-------------------------------------------------
1e-05    0.999999972838578    1.000000027161422
1e-07    0.999999961581419    1.000000038418581
1e-09    1.000000002000000    0.999999998000000
1e-11    1.000000000020000    0.999999999980000
1e-13    1.000000000000200    0.999999999999800
1e-15    1.000000000000002    0.999999999999998
1e-17    1.000000000000000    1.000000000000000


In [61]:
# assign either 'answer = True' OR 'answer = False'
# YOUR CODE HERE
answer = False

3) In the implementation of Gaussian elimination (in the form of $LU$ composition), we zero out the coefficient `A[i,j]` below the pivot `A[i,i]` by performing a row-operation with the scalar multiplier `m = A[i,j]/A[i,i]` so, if that coefficient has larger magnitude than the pivot, the multiplier has magnitude greater than 1. As illustrated in Problem 2, a large multiplier can cause numerical errors to grow leading to loss of precision in the solution.

Having implemented `swap_rows()` in Problem 1, we can now avoid this problem with __partial pivoting__. The basic idea is to swap the pivot row with whichever row beneath it has the largest coefficient in the current pivot column. After swapping this row into the pivot position, all multipliers in the row ops to zero out the entries below the diagonal will have magnitude less than 1.

To implement a partial pivoting approach in the language of matrices, we will again think in terms of matrix factorization. We saw that basic Gaussian elimination can be thought of in terms of an equivalent $LU$ factorization where the Upper triangular matrix $U$ holds the triangular matrix obtained by elimination, and the lower triangular matrix $L$ records the multipliers in the elementary row operations.

For partial pivoting, we also need to record row swaps leading to the factorization $A = PLU$ where P is the __permutation matrix__ (a square matrix that is all zeros except for a single 1 in each row and column - alternatively think of $P$ as an identity matrix that has been subjected to row swaps). 

Your task in this problem is to implement $PLU$ factorization of a square input matrix `A`. The approach is very similar to what was done in $LU$ factorization, but now with the additional factor $P$ to record swaps. Here are the steps:

0. Get the size of the $A$ matrix and check that it is square.

1. Initialize `P` and `L` as identity matrices and `U` as a copy of `A`

2. Loop over the diagonal elements `A[i,i] for i in range(m)`:

    a. Identify row `imax` having the largest magnitude entry in sub-column `i` (at or below the main diagonal)
    
    b. Swap rows `imax` and `i` in $U$ and in $P$
    
    d. Perform row ops on $U$ to zero out the column below the pivot and record multipliers in `L`.
    
3. Return `P, L, U`

Insert your code for $PLU$ factorization in the cell below. (The code for `LU_factor` is included for reference.)

In [8]:
def LU_factor(A):
    m,n = A.shape
    if m != n:
        print("WARNING: Non-square input matrix")
        return
    mult = 0
    U = np.copy(A) #make a copy of the array
    #Note that U=A just makes another name for A, not a new copy of the array
    L = np.eye(n) #numpy's name for the identity matrix is "eye"
    for i in range(n): # for each row i
        for j in range(i+1,n): # for each row below row i
            mult = U[j,i]/U[i,i]
            L[j,i] = mult
            for k in range(i,n): # for each entry beyond the i^th diagonal entry
                U[j,k] = U[j,k] - mult*U[i,k] # for entries to the right, subtract multiple of term in row i         
    return L,U

def PLU_factor(A):
    """
    perform elementary row operation to swap rows i0 and i1
    
    Args:
        A:     2D numpy array representing a matrix
        i0,i1: integer row indices
    Returns:
        P,L,U: nxn numpy arrays - permutation, lower triangular, upper triangular
    """
    # Basing on stackoverflow
    m,n = A.shape
    if m != n:
        print("WARNING: Non-square input matrix")
        return
    mult = 0
    U = np.copy(A).astype(np.float)
    L = np.zeros((n, n), dtype = 'float') 
    P = np.eye(n).astype(np.float)


    for i in range(n):
        imax = i
        comp = abs(U[i, i])
        for j in range(i, n):
            if abs(U[j, i]) > comp:
                imax = j
                comp = abs(U[j, i])
                
        swap_rows(U, imax, i)
        swap_rows(P, imax, i)
        swap_rows(L, imax, i)
        
        for j in range(i + 1, n):
            mult = U[j, i] / U[i, i]
            L[j, i] = mult 
            U[j, :] -= mult * U[i, :]
            
    L = L + np.eye(n)
    
    return(P.T, L, U)

In [9]:
from scipy.linalg import lu
A = np.array([[2,3,4],[1,1,1],[7,5,1]], dtype = np.float64)
P,L,U = PLU_factor(A)
P0,L0,U0 = lu(A)
assert_(np.allclose(P,P0))
assert_(np.allclose(L,L0))
assert_(np.allclose(U,U0))

4a) The $LU$ solver introduced the intermediate unknown vector $y$ and called the triangular solvers to solve $Ly=b$ and then $Ux=y$. State the corresponding triangular equations that need to be solved by the $PLU$ solver.

YOUR ANSWER HERE: Ly =(P^T)b & Ux = y 

4b) Write a linear solver that utilizes $PLU$ factorization. The cell below has the code for solving $Ax=b$ using $LU$ factorization for reference. You can use the triangular solver functions as is, but modify the general solver as necessary to use $PLU$ instead of $LU$.

Insert your code for your $PLU$ solver in the cell below:

In [10]:
def LU_solve(A,b):
    L,U = LU_factor(A)
    y = lower_tri_solve(L,b)
    x = upper_tri_solve(U,y)
    return x, y

def PLU_solve(A,b):
    """
    solve linear system Ax=b using PLU factorization (Gaussian elimination with pivoting)
    
    Args:
        A: 2D numpy array representing a matrix
        b: 1D numpy array of right-hand side values
    Returns:
        x: 1D numpy array corresponding to solution
    """

    P,L,U = PLU_factor(A)
    y = lower_tri_solve(L,np.dot(P.T,b))
    x = upper_tri_solve(U,y)

    return x

In [11]:
b = np.array([24,7,17])
x0 = np.array([3,-2,6])
x = PLU_solve(A,b)
assert_(np.allclose(x, x0))


7) Write code to implement the matrix iteration scheme for computing the dominant eigenvalue/eigenvector of a square matrix.

In [12]:
def dominant_eigen_iteration(A, u0, tol, max_iters):
    """
    compute dominant eigenvctor and eigenvalue for square matrix
    
    Args:
        A:  nxn numpy array representing a matrix
        u0: initial estimate of eigenvector (1D numpy array)
        tol: float relaitve error termination criterion
        max_iters: integer iteration count termination criterion
    Returns:
        P,L,U: nxn numpy arrays - permutation, lower triangular, upper triangular
    """

    zero_1dmatrix = np.zeros(1,dtype = np.float64)
    for i in range(0,max_iters):
        answer = np.dot(A,u0)
        elements = answer.flat[abs(answer).argmax()]
        zero_1dmatrix = np.append(zero_1dmatrix,[elements])
        u0 = answer/elements
        error = abs(zero_1dmatrix[i+1]-zero_1dmatrix[i])
        if error <= tol :
            normal = np.linalg.norm(u0)
            n_array = u0/normal
            return elements,n_array

    return elements,n_array

In [13]:
A = np.array([[2,-1,0,0],[-1,2,-1,0],[0,-1,2,-1],[0,0,-1,2]])
u0 = np.array([1,1,2,1])
tol = 0.001
max_iters = 100
w,v = dominant_eigen_iteration(A, u0, tol, max_iters) 
print(w)
print(v)

3.6161710154384368
[-0.37305442  0.60230916 -0.6006916   0.37043715]


In [14]:
A = np.array([[2,-1,0,0],[-1,2,-1,0],[0,-1,2,-1],[0,0,-1,2]])
u0 = np.array([1,1,2,1])
tol = 0.001
max_iters = 100
w,v = dominant_eigen_iteration(A, u0, tol, max_iters)
expected_v = np.array([-0.3717, 0.6015, -0.6015, 0.3717])
expected_w = 3.618
assert_(np.allclose(v, expected_v, atol=2e-01) or np.allclose(-v, expected_v, atol=2e-01))
assert_(abs(w - expected_w) < 1e-2)
